In [1]:
from datetime import datetime
import numpy as np
import math
from scipy.optimize import *
import pandas as pd
from matplotlib import pyplot as plt
from lmfit import Model

### Inputs and Parameters

In [3]:
#Units

nano = 1e-9
micro = 1e-6
milli = 1e-3

#Hologram_Recording_Parameters
lambda_w = 405*nano
lambda_r = 633*nano
theta_bragg = np.deg2rad(11.68)

#Materials_Parameters - changes based on the material being used
n_mono = float(1.5)                        # Monomer index
nP = float(1.6)                            # Polymer index
nM = float(1.48)                           # Matrix index
sigma = float(0.05)                        # Full conv shrinkage
phimo = float(0.3)                         # Formulation monomer volume fraction
mu = 0                                     # Attenuation coefficient, [1/m]
Z = 50*micro                               # Sample thickness


#Exposure Parameters
LAMBDA = lambda_w/(2*np.sin(theta_bragg))          # Grating period of the hologram at 12 degree bragg angle normal to the surface, change the angle if bragg angke is different
K_inc_air =  (2*np.pi)/lambda_w                    # Plane wave of the incident beam - in air
K_ref_air =  (2*np.pi)/lambda_w                    # Plane wave of the reference beam  - in air
K_read_air = (2*np.pi)/lambda_r                    # Plane wave of the reading beam - from recording beam in air
K_inc =  (2*np.pi*nP)/lambda_w                     # Plane wave of the incident beam  - in polymer
K_diff = (2*np.pi*nP)/lambda_r                     # Plane wave of the diffracted beam - from recording beam in polymer
K = (2*np.pi)/(LAMBDA)                             # Wavevector of the beams, capital lambda is the period
w_o_wrt = 100*micro                                # Waist radius of writing beams
w_o_red = 20*micro                                 # Waist radius of reading beams
Nt = 20                                            # Number of BPM time steps
ExpT = 0.05                                        # Total exposure time multiplied by peak (NOT mean) polymerization rate
Gm = 10                                            # Diff_monomer / (Lambda^2 R_P) where R_P is propagation rate (depends on initiator, intensity & reaction rate)
Gr = 0                                             # Diff_radical / (Lambda^2 R_A) where R_A is attachment rate of mobile radicals to immobile matrix

#Controls
def Bragg_condition():
    a_w = ((K/2)/K_inc_air)
    a_r = ((K/2)/K_read_air)
    theta_bragg_w = np.arcsin(a_w)            #Bragg Angle of the writing beam
    theta_bragg_r = np.arcsin(a_r)            #Bragg Angle of the recording beam
    theta_w = np.rad2deg(theta_bragg_w)
    theta_r = np.rad2deg(theta_bragg_r)
    Bragg_Condition = "Bragg Angle of the writing beam: ", theta_w, "Bragg Angle of the reading beam: ", theta_r
    return Bragg_Condition

def save_file():
    Parameters = Bragg_condition()
    return Parameters
output = save_file()
file = open("Data-s%"+ str(datetime.today().strftime("%Y-%m-%d--%H_%p")) + ".csv","w")
file.write(str(output))
file.close()

### Numerical Calculations

### Extracting and Cleaning Data

In [ ]:
data_exp1 = "Data_num1.txt"
data_exp2 = "Data_num2.txt"
data_exp3 = "Data_num3.txt"
data_exp4 = "Data_num4.txt"
data_exp5 = "Data_num5.txt"
data_exp6 = "Data_num6.txt"
data_exp7 = "Data_num7.txt"
data_exp8 = "Data_num8.txt"
data_exp9 = "Data_num9.txt"
data_exp10 = "Data_num10.txt"
data_exp11 = "Data_num11.txt"
data_exp12 = "Data_num12.txt"
data_exp13 = "Data_num13.txt"
data_exp14 = "Data_num14.txt"
data_exp15 = "Data_num15.txt"
data_exp16 = "Data_num16.txt"
data_exp17 = "Data_num17.txt"
data_exp18 = "Data_num18.txt"
data_exp19 = "Data_num19.txt"
data_exp20 = "Data_num20.txt

Data = data_exp1
df = pd.read_csv(data)
df.columns = ['XValue', 'Time', 'Intensity', 'Wavelength', 'Incidence', 'Diffracted', 'DE_num', 'stage_angle', 'Exposure_Time']
n = len(df.stage_angle) #number of angles considered
Bragg_angle_ext = np.deg2rad(18) #Bragg angle for red beam, it is 12 for blue beam #EXTERNAL BRAGG
Bragg_angle_int = np.arcsin((1/1.5)*(np.sin(Bragg_angle_ext)))
df.stage_angle = np.deg2rad(df.stage_angle) #convert angles to radians to make them suitable for trigonometry functions
df["ext_angle"] = Bragg_angle_ext + df.stage_angle
df["int_angle"] = np.arcsin((1/1.5)*(np.sin(df.ext_angle))) #Snell's law for the external angle, to change to internal angles

df['Delta_theta_ext'] = (df.ext_angle - Bragg_angle_ext) #get the deviation from bragg angle
df['Delta_theta_int'] = (df.int_angle - Bragg_angle_int)

phi = 90
Delta_Lambda = 0

### Bragg Selectivity using Kogelnik - Numeric and Kogelnik Theory

In [ ]:
x = df.int_angle
y = df.DE_num

def func(x, Delta_n, L):
    nu = ((math.pi*Delta_n*L)/(lambda_r)*((((np.cos(Bragg_angle_int)**2))-((K*np.cos(phi)*np.cos(Bragg_angle_int))/K_diff))**0.5))
    xi = (((df.Delta_theta_int*K*np.sin(phi-Bragg_angle_int)-((Delta_Lambda*K**2)/(4*np.pi*nP)))*L)/2*((np.cos(df.int_angle))-(K*np.cos(phi))/K_diff))
    df["xi"] = xi
    return ((np.sin(((nu**2)+(df.xi**2))**0.5)**2)/(1+(df.xi**2)/(nu**2)))

def Bragg_Selectivity():
    init_vals = [3e-3, 21e-6]  # for [Delta_n,cen, L]
    best_vals, covar = curve_fit(func, x, y, p0=init_vals)
    dn = best_vals[0]
    d = best_vals[1]
    return (dn, " ", d)

Bragg_Selectivity()
f=open("Data.txt".format(datetime.today()), "a+")
file.write("Index Modulation (Theory): " , dn)
file.write("Photopolymer Thickness (Theory): " , d)
file.close()

#Plot and fit data
gmodel = Model(func)
f=open("Data.txt".format(datetime.today()), "a+")
file.write('parameter names: {}'.format(gmodel.param_names))
file.write('independent variables: {}'.format(gmodel.independent_vars))
file.close()

params = gmodel.make_params(Delta_n=3e-3, L=21e-6)
result = gmodel.fit(y, x=x, Delta_n=3e-3, L=21e-6)
f=open("Data.txt".format(datetime.today()), "a+")
file.write(result.fit_report())
file.close()

### Figures and Plots

In [ ]:
#Bragg Selectivity - Angle Detuning


#plot - ensure parameters are rightly named
df.int_angle = np.rad2deg(df.int_angle)
df.ext_angle = np.rad2deg(df.ext_angle)
plt.plot(df.ext_angle, y*100, 'b.', label = 'Numeric')
plt.plot(df.ext_angle, result.init_fit*100, 'k--', label='initial fit')
plt.plot(df.ext_angle, result.best_fit*100, 'r-', label='best fit')
plt.title("Diffraction Efficiency vs θ_ext")
plt.xlabel("θ_ext (Degree)")
plt.ylabel("Diffraction Efficiency %")
plt.legend(loc='best')
plt.show()
plt.savefig('filename.png')

In [ ]:
# Shrinkage


In [ ]:
#Index Modulation and Exposure Time
